<a href="https://colab.research.google.com/github/Ftajedini/Github_Tutorial/blob/main/exteract_parcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
#!pip install arcgis
#!pip install timm
#!pip install Torch
#!pip install fastai
#!pip3 install git+https://github.com/fastai/fastai.git

  Cloning https://github.com/fastai/fastai.git to /tmp/pip-req-build-50mk2_41
  Running command git clone --filter=blob:none --quiet https://github.com/fastai/fastai.git /tmp/pip-req-build-50mk2_41
  Resolved https://github.com/fastai/fastai.git to commit 96f0e2027c8fd82eef40984faf14c1c81e0eb031
  Preparing metadata (setup.py) ... done


In [3]:
from arcgis.gis import GIS
gis = GIS()
gis_ent = GIS('https://pythonapi.playground.esri.com/portal/')

In [4]:
training_raster = gis_ent.content.get('c3bb538699964697bae61570a77768cb')
training_raster

<Item title:"la_county_rgb_raster" type:Imagery Layer owner:api_data_owner>

In [5]:
input_feature_class = gis.content.get('d64f11aea3ae47c19084f21c204ba318')
input_feature_class

<Item title:"input_features_lulc" type:Feature Layer Collection owner:api_data_owner>

In [6]:
instance_feature_class = gis.content.get('22b83520db8f42ea98a5279b71684afd')
instance_feature_class

<Item title:"instance_features_buildings_cars" type:Feature Layer Collection owner:api_data_owner>

In [18]:
import os
import zipfile
from pathlib import Path
from fastai.basics import *
from arcgis.learn import prepare_data, MaXDeepLab

In [22]:
import fastai
print(fastai.__version__)
from fastai import *
from fastai.vision import *

2.7.13


In [23]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.backends.cudnn.enabled)

2.1.0+cu118
True
True


In [11]:
#Get training data
training_data = gis.content.get('69544aca740d42d3a3e36f7ad5ddb485')
training_data

<Item title:"panoptic_segmentation_training_data" type:Image Collection owner:api_data_owner>

In [12]:
filepath = training_data.download(file_name=training_data.name)

In [15]:
import zipfile
with zipfile.ZipFile(filepath, 'r') as zip_ref:
    zip_ref.extractall(Path(filepath).parent)

In [24]:
data_path = Path(os.path.join(os.path.splitext(filepath)[0]))

##Prepare data

In [20]:
data = prepare_data (data_path, batch_size=4, chip_size=512, n_masks=30)

Exception: ignored

##Visualize training data

In [ ]:
data.show_batch(alpha=0.8)

In [ ]:
lr = model.lr_find()
lr

##Fit the model

In [ ]:
model.fit(20, lr=lr)

In [ ]:
model.plot_losses()

#Visualize results in validation set

In [ ]:
model.show_results(alpha=0.8)

#Accuracy assessment

In [ ]:
model.panoptic_quality()

In [ ]:
model.save("la_panoptic_512px_ep21_lr001", publish=True)

#Deploy the model

In [25]:
trained_model = gis.content.get('4ef21be1d9fb4ced9eacb306558e8edd')
trained_model

<Item title:"la_panoptic_512px_ep21_lr001" type:Deep Learning Package owner:api_data_owner>

#Results

In [26]:
fc = gis_ent.content.get('3b5033eb16044a089795682256e819ae')
fc

<Item title:"panoptic_resultsCP" type:Imagery Layer owner:api_data_owner>

In [27]:
ic = gis.content.get('ca935d3e3e614e2392485e07ab56b97f')
ic

<Item title:"panoptic_results" type:Feature Layer Collection owner:api_data_owner>

#Visualize results using map widgets

In [29]:
map1 = gis_ent.map()
map1.add_layer(inference_raster)
map2 = gis.map()
map2.add_layer(fc)
map2.add_layer(ic)

NameError: ignored

In [ ]:
map2.sync_navigation(map1)

In [ ]:
from ipywidgets import HBox, VBox, Label, Layout

In [ ]:
hbox_layout = Layout()
hbox_layout.justify_content = 'space-around'

hb1=HBox([Label('Raster'),Label('Results')])
hb1.layout=hbox_layout

In [ ]:
VBox([hb1,HBox([map1,map2])])